In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import itertools
from scipy.stats import pearsonr, spearmanr
from IPython.display import display

In [2]:
cleaned_dfs = []
for df, setting, dataset in zip(
    [
        pd.read_csv("/root/similaritybench/experiments/results/nlp_aug_sst2.csv"),
        pd.read_csv("/root/similaritybench/experiments/results/nlp_mem_sst2.csv"),
        pd.read_csv("/root/similaritybench/experiments/results/nlp_shortcut_sst2.csv"),
        pd.read_csv("/root/similaritybench/experiments/results/nlp_shortcut_mnli.csv")
    ],
    ["aug", "mem", "sc", "sc"],
    ["sst2", "sst2", "sst2", "mnli"]
):
    data = df.loc[2:].copy().reset_index(drop=True)
    data.columns = ["Similarity Measure", "Violation Rate", "AUPRC"]
    data["Architecture"] = "BERT-L"
    data.loc[:, "Violation Rate"] = data.loc[:, "Violation Rate"].astype(float)
    data.loc[:, "AUPRC"] = data.loc[:, "AUPRC"].astype(float)
    data = data.melt(
        id_vars=["Similarity Measure", "Architecture"],
        value_vars=["Violation Rate", "AUPRC"],
        var_name="Quality Metric",
        value_name="Score",
    )
    data["Setting"] = setting
    data["Dataset"] = dataset

    cleaned_dfs.append(data)

data = pd.concat(cleaned_dfs).reset_index(drop=True)
data.head()

Similarity Measure Architecture  Quality Metric     Score Setting  \
0  AlignedCosineSimilarity       BERT-L  Violation Rate  0.408722     aug   
1                      CKA       BERT-L  Violation Rate  0.342889     aug   
2  ConcentricityDifference       BERT-L  Violation Rate  0.467389     aug   
3      DistanceCorrelation       BERT-L  Violation Rate  0.345522     aug   
4   EigenspaceOverlapScore       BERT-L  Violation Rate  0.516689     aug   

  Dataset  
0    sst2  
1    sst2  
2    sst2  
3    sst2  
4    sst2

In [3]:
data_plot = data.copy()
data_plot.loc[:, "Quality Metric"] = data_plot.loc[:, "Quality Metric"].map(
    {"Violation Rate": "Conformity Rate", "AUPRC": "AUPRC"}, na_action="ignore"
)
data_plot.loc[data_plot["Quality Metric"] == "Conformity Rate", "Score"] = data_plot.loc[
        data_plot["Quality Metric"] == "Conformity Rate", "Score"
].map(lambda x: 1 - x)
data_sorted = data_plot.sort_values(by=["Score", "Quality Metric"], ascending=True)

sns.set(style="darkgrid")
g = sns.catplot(
    data=data_sorted,
    x="Similarity Measure",
    y="Score",
    hue="Quality Metric",
    row="Setting",
    kind="bar",
    aspect=3,
    palette="Set2",
    legend=True,
    legend_out=False,
)
g.set_xticklabels(rotation=45, ha="right")

In [6]:
# Select score or rank
ycol = "Score" if True else "rank"

ranks = data.groupby(["Setting", "Quality Metric"])["Score"].rank()
data["rank"] = ranks
for qmeasure in data["Quality Metric"].unique():
    print(qmeasure)
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(
            data[data["Quality Metric"] == qmeasure]
            .groupby(["Similarity Measure", "Quality Metric", "Setting", "Dataset"])[ycol]
            .agg(["mean", "min", "max"])
            .sort_values(["Setting", "mean"])
        )

Violation Rate


mean  \
Similarity Measure                        Quality Metric Setting Dataset             
CKA                                       Violation Rate aug     sst2     0.342889   
SVCCA                                     Violation Rate aug     sst2     0.343289   
DistanceCorrelation                       Violation Rate aug     sst2     0.345522   
RSA                                       Violation Rate aug     sst2     0.366289   
OrthogonalAngularShapeMetricCentered      Violation Rate aug     sst2       0.3919   
OrthogonalProcrustesCenteredAndNormalized Violation Rate aug     sst2       0.3919   
AlignedCosineSimilarity                   Violation Rate aug     sst2     0.408722   
ProcrustesSizeAndShapeDistance            Violation Rate aug     sst2     0.411778   
JaccardSimilarity                         Violation Rate aug     sst2     0.422944   
SecondOrderCosineSimilarity               Violation Rate aug     sst2     0.425756   
RankSimilarity                            Violation Rate aug     sst2     0.441833   
ConcentricityDifference                   Violation Rate aug     sst2     0.467389   
IMDScore                                  Violation Rate aug     sst2     0.484533   
SoftCorrelationMatch                      Violation Rate aug     sst2     0.485978   
HardCorrelationMatch                      Violation Rate aug     sst2     0.490944   
PWCCA                                     Violation Rate aug     sst2     0.494895   
UniformityDifference                      Violation Rate aug     sst2     0.501378   
MagnitudeDifference                       Violation Rate aug     sst2     0.510133   
GeometryScore                             Violation Rate aug     sst2     0.511578   
PermutationProcrustes                     Violation Rate aug     sst2     0.515867   
EigenspaceOverlapScore                    Violation Rate aug     sst2     0.516689   
Gulp                                      Violation Rate aug     sst2     0.531089   
RSMNormDifference                         Violation Rate aug     sst2          1.0   
ConcentricityDifference                   Violation Rate mem     sst2        0.048   
AlignedCosineSimilarity                   Violation Rate mem     sst2       0.1018   
ProcrustesSizeAndShapeDistance            Violation Rate mem     sst2       0.1592   
UniformityDifference                      Violation Rate mem     sst2       0.2842   
DistanceCorrelation                       Violation Rate mem     sst2       0.3082   
SVCCA                                     Violation Rate mem     sst2       0.3164   
SoftCorrelationMatch                      Violation Rate mem     sst2       0.3258   
CKA                                       Violation Rate mem     sst2       0.3268   
HardCorrelationMatch                      Violation Rate mem     sst2        0.354   
IMDScore                                  Violation Rate mem     sst2       0.3558   
RSA                                       Violation Rate mem     sst2       0.3584   
OrthogonalAngularShapeMetricCentered      Violation Rate mem     sst2       0.3626   
OrthogonalProcrustesCenteredAndNormalized Violation Rate mem     sst2       0.3626   
PermutationProcrustes                     Violation Rate mem     sst2        0.376   
JaccardSimilarity                         Violation Rate mem     sst2       0.3848   
MagnitudeDifference                       Violation Rate mem     sst2         0.39   
SecondOrderCosineSimilarity               Violation Rate mem     sst2       0.4152   
RankSimilarity                            Violation Rate mem     sst2        0.418   
EigenspaceOverlapScore                    Violation Rate mem     sst2       0.4594   
PWCCA                                     Violation Rate mem     sst2     0.495704   
Gulp                                      Violation Rate mem     sst2       0.4986   
GeometryScore                             Violation Rate mem     sst2       0.5112   
RSMNormDifference                   

AUPRC


mean  \
Similarity Measure                        Quality Metric Setting Dataset             
Gulp                                      AUPRC          aug     sst2     0.100955   
RSMNormDifference                         AUPRC          aug     sst2     0.101124   
PermutationProcrustes                     AUPRC          aug     sst2     0.101906   
EigenspaceOverlapScore                    AUPRC          aug     sst2     0.103331   
HardCorrelationMatch                      AUPRC          aug     sst2     0.104825   
MagnitudeDifference                       AUPRC          aug     sst2     0.106836   
UniformityDifference                      AUPRC          aug     sst2     0.107856   
SoftCorrelationMatch                      AUPRC          aug     sst2     0.111305   
GeometryScore                             AUPRC          aug     sst2      0.11419   
RankSimilarity                            AUPRC          aug     sst2     0.116057   
ConcentricityDifference                   AUPRC          aug     sst2     0.116144   
IMDScore                                  AUPRC          aug     sst2      0.12023   
JaccardSimilarity                         AUPRC          aug     sst2     0.120932   
SecondOrderCosineSimilarity               AUPRC          aug     sst2     0.132104   
ProcrustesSizeAndShapeDistance            AUPRC          aug     sst2     0.133479   
AlignedCosineSimilarity                   AUPRC          aug     sst2     0.133663   
OrthogonalProcrustesCenteredAndNormalized AUPRC          aug     sst2     0.155229   
OrthogonalAngularShapeMetricCentered      AUPRC          aug     sst2     0.155239   
SVCCA                                     AUPRC          aug     sst2     0.202352   
DistanceCorrelation                       AUPRC          aug     sst2     0.202842   
RSA                                       AUPRC          aug     sst2     0.203099   
CKA                                       AUPRC          aug     sst2     0.205222   
PWCCA                                     AUPRC          aug     sst2          NaN   
RSMNormDifference                         AUPRC          mem     sst2     0.090909   
GeometryScore                             AUPRC          mem     sst2     0.104324   
RankSimilarity                            AUPRC          mem     sst2     0.127127   
EigenspaceOverlapScore                    AUPRC          mem     sst2     0.128683   
JaccardSimilarity                         AUPRC          mem     sst2     0.129887   
SecondOrderCosineSimilarity               AUPRC          mem     sst2     0.140627   
Gulp                                      AUPRC          mem     sst2     0.149911   
MagnitudeDifference                       AUPRC          mem     sst2      0.15187   
RSA                                       AUPRC          mem     sst2     0.154634   
OrthogonalAngularShapeMetricCentered      AUPRC          mem     sst2     0.162086   
OrthogonalProcrustesCenteredAndNormalized AUPRC          mem     sst2     0.162086   
HardCorrelationMatch                      AUPRC          mem     sst2     0.202568   
CKA                                       AUPRC          mem     sst2     0.218001   
DistanceCorrelation                       AUPRC          mem     sst2     0.221519   
SoftCorrelationMatch                      AUPRC          mem     sst2     0.226598   
PermutationProcrustes                     AUPRC          mem     sst2     0.232547   
SVCCA                                     AUPRC          mem     sst2     0.256518   
IMDScore                                  AUPRC          mem     sst2     0.257677   
UniformityDifference                      AUPRC          mem     sst2     0.333816   
ProcrustesSizeAndShapeDistance            AUPRC          mem     sst2     0.448783   
AlignedCosineSimilarity                   AUPRC          mem     sst2     0.512842   
ConcentricityDifference                   AUPRC          mem     sst2     0.737925   
PWCCA                               

In [5]:
qual_metrics = data["Quality Metric"].unique()
corrs = np.full(shape=(2, len(qual_metrics), len(qual_metrics)), fill_value=np.nan)  # two elems in first dim for pearson and spearman
metric_to_idx = {m: i for i, m in enumerate(qual_metrics)}

for m1, m2 in itertools.combinations(qual_metrics, r=2):
    scores1 = data.loc[data["Quality Metric"] == m1, "Score"].reset_index(drop=True)
    scores2 = data.loc[data["Quality Metric"] == m2, "Score"].reset_index(drop=True)
    if len(scores1) != len(scores2):
        print(f"Unequal length! {m1, m2}")
        continue

    nan_idx = scores1.isna() | scores2.isna()
    vals1 = scores1[~nan_idx].values
    vals2 = scores2[~nan_idx].values
    p = pearsonr(vals1, vals2)
    s = spearmanr(vals1, vals2)

    i = metric_to_idx[m1]
    j = metric_to_idx[m2]
    corrs[0, i, j] = p.statistic
    corrs[0, j, i] = p.statistic
    corrs[1, i, j] = s.statistic
    corrs[1, j, i] = s.statistic

print("Pearson", corrs[0])
print("Spearman", corrs[1])

Pearson [[        nan -0.67489882]
 [-0.67489882         nan]]
Spearman [[        nan -0.94864912]
 [-0.94864912         nan]]
